In [1]:
DBPATH = './28_test.db'

import time
import numpy as np

import qcodes as qc
from qcodes import load_or_create_experiment, Measurement, Parameter

qc.config.core.db_location = DBPATH
qc.initialise_database()

from pytopo.sweep import sweep

In [2]:
x = Parameter('x')
y = Parameter('y')
z = Parameter('z')
data = Parameter('data')

station = qc.Station(x, y, z, data)

In [3]:
xvals = np.arange(5).astype(float)
yvals = np.arange(5).astype(float)
zvals = np.arange(5).astype(float)
xx, yy, zz = np.meshgrid(xvals, yvals, zvals, indexing='ij')
dd = xx + yy + zz

def soft_3d_sweep():
    for x, y, z, data in zip(xx.reshape(-1), yy.reshape(-1), zz.reshape(-1), dd.reshape(-1)):
        yield x, y, z, data

exp = load_or_create_experiment('soft_3d_sweep_test', sample_name='no sample')

meas = Measurement(exp, station)
meas.register_parameter(x)
meas.register_parameter(y)
meas.register_parameter(z)
meas.register_parameter(data, setpoints=(x, y, z))
meas.write_period = 2

with meas.run() as datasaver:
    for xval, yval, zval, dataval in soft_3d_sweep():
        print(f"x = {xval}, y = {yval}, z={zval} {10*' '}", end='\r')
        datasaver.add_result(
            (x, xval),
            (y, yval),
            (z, zval),
            (data, dataval),
        )
        time.sleep(0.5)

Starting experimental run with id: 4


KeyboardInterrupt: 

In [21]:
import pandas as pd

In [37]:
arr = np.array([
    [0, 0, 0],
    [1, np.nan, np.nan],
    [np.nan, np.nan, np.nan],
])

In [43]:
pd.DataFrame(arr).interpolate(axis=1, method='linear')

,0,1,2
0,0.0,0.0,0.0
1,1.0,1.0,1.0
2,NaN,NaN,NaN


In [95]:
arr = np.arange(16.0).reshape(4,4)
arr[-1] = np.nan
arr[:,-1] = np.nan
arr = np.ma.masked_invalid(arr)
# arr.fill_value = np.nan
arr

masked_array(
  data=[[0.0, 1.0, 2.0, --],
        [4.0, 5.0, 6.0, --],
        [8.0, 9.0, 10.0, --],
        [--, --, --, --]],
  mask=[[False, False, False,  True],
        [False, False, False,  True],
        [False, False, False,  True],
        [ True,  True,  True,  True]],
  fill_value=1e+20)

In [97]:
np.mean(arr, axis=1)

masked_array(data=[1.0, 5.0, 9.0, --],
             mask=[False, False, False,  True],
       fill_value=1e+20)

In [103]:
arr.toarray()

AttributeError: 'MaskedArray' object has no attribute 'toarray'